In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib
# matplotlib.artist.getp(fig.patch) # https://matplotlib.org/stable/tutorials/intermediate/artists.html#sphx-glr-tutorials-intermediate-artists-py
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
# usually this notebook is loaded as top-level-module and relative import of chsimpy does not work.
# so we provide the path to the chsimpy package manually
import pathlib
import sys

try:
    import chsimpy
except ImportError:
    _parentdir = pathlib.Path("./").resolve().parent
    sys.path.insert(0, str(_parentdir))
    import chsimpy
    #sys.path.remove(str(_parentdir))

# auto reload if chsimpy code changed
%load_ext autoreload
%autoreload 2

In [ ]:
dirfiles = '../_output/paper-1k'
rawfile = f"{dirfiles}/experiment-17022023-114325-raw.csv"
sysinfofile = f"{dirfiles}/experiment-17022023-114325-sysinfo.csv"
aggfile = f"{dirfiles}/experiment-17022023-114325-agg.csv"

In [ ]:
dfagg = pd.read_csv(aggfile, index_col=0)
dfraw = pd.read_csv(rawfile, index_col=['id'])
dfraw = dfraw.drop(columns=dfraw.columns[0]) # drops unnamed column
sysinfo = pd.read_csv(sysinfofile, header=None).T

In [ ]:
dfsys = sysinfo.iloc[0].str.split('=')
keys = [ x[0] for x in dfsys]
values = [ x[1].replace("'", '') for x in dfsys]
dfsys = pd.DataFrame(values, index=keys)

In [ ]:
dfagg

In [ ]:
dfraw

In [ ]:
cacb = np.abs(dfraw.ca-dfraw.cb)
#
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 10))
#
# nrow, ncol, index
axs[0,0].plot(dfraw.A0, cacb, '.', ms=2)
axs[0,0].set_xlabel('A0')
axs[0,0].set_ylabel('|c_a - c_b|')

axs[0,1].plot(dfraw.A1, cacb, '.', ms=2)
axs[0,1].set_xlabel('A1')
axs[0,1].set_ylabel('|c_a - c_b|')

axs[1,0].plot(dfraw.A0, dfraw.tsep, '.', ms=2, c='purple')
axs[1,0].set_xlabel('A0')
axs[1,0].set_ylabel('t_sep')

axs[1,1].plot(dfraw.A1, dfraw.tsep, '.', ms=2, c='purple')
axs[1,1].set_xlabel('A1')
axs[1,1].set_ylabel('t_sep')

axs[2,0].plot(cacb, dfraw.tsep, '.', ms=2, c='y')
axs[2,0].set_xlabel('|c_a - c_b|')
axs[2,0].set_ylabel('t_sep')

dfcacb = pd.melt(dfraw[['cb','ca']])
sns.boxplot(y="variable", x="value", data=dfcacb, ax=axs[2,1])
#axs[2,1].plot(cacb, dfraw.tsep-dfraw.tau0, '.', ms=2, c='y')
axs[2,1].set_xlabel('c_* for T = 923.15K')
axs[2,1].set_ylabel('')

display(fig)
matplotlib.pyplot.close()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(8, 5), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
#
# nrow, ncol, index
sns.kdeplot(dfraw.tsep, ax=axs[0])
axs[0].set_xlabel('')
axs[0].set_ylabel('dist')

sns.boxplot(dfraw.tsep, orient='h', ax=axs[1])
axs[1].set_xlabel('iterations')
axs[1].set_ylabel('')
axs[1].set(yticklabels=[])

display(fig)
matplotlib.pyplot.close()

In [ ]:
from IPython.display import Math
import sympy as sym
from sympy.plotting import plot as symplot
sym.init_printing()  # automatically enable the best printer available in your environment.
import seaborn as sns
sns.set()
sns.set_style("whitegrid", {'grid.linestyle': '--'})

In [ ]:
R = 0.0083144626181532
B = 12.81
Ts = 923.15 + np.arange(-100,200,25)
count = Ts.shape[0]
x0 = np.linspace(0.7,0.825,count)
x1 = np.linspace(0.0001,0.01,count)
ca = np.zeros(count)
cb = np.zeros(count)
for i in range(0, count):
    T = Ts[i]
    A0 = 186.0575 - 0.3654*T
    A1 = 43.7207 - 0.1401*T
    print(T)
    cacb = chsimpy.utils.get_miscibility_gap(R=R, T=T, B=B, A0=A0, A1=A1,
                                             xlower=x0[i], xupper=1.0-x1[i], prec=10)
    ca[i] = cacb[0]
    cb[i] = cacb[1]
    #print(T, ca[i], cb[i])

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
#
# nrow, ncol, index
axs.plot((ca,cb), (Ts,Ts), 'o', ms=4, color='black')
axs.set_xlim((0.7, 1.0))
display(fig)
matplotlib.pyplot.close()

In [ ]:
R = 0.0083144626181532
T = 923.15
B = 12.81
A0 = -151.26151  # 186.0575 - 0.3654*T;  % in kJ / mol
A1 = -85.612615  # 43.7207 - 0.1401*T;  % in kJ / mol
RTB=R*T*B
x = sym.Symbol('x', real=True)
c = x  # c acts a placeholder for x, x1, x2, ..

E = (R*T * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c)) #+ 0.5*eps2*du2

cacb = chsimpy.utils.get_miscibility_gap(R=R, T=T, B=B, A0=A0, A1=A1)
ca = cacb[0]
cb = cacb[1]

In [ ]:
jitter_Arellow = 0.995
jitter_Arelhigh = 1.005
runs = 100

rng = np.random.default_rng(2023)
randv = rng.uniform(jitter_Arellow, jitter_Arelhigh, size=(2, runs))
seta0 = A0 * randv[0]
seta1 = A1 * randv[1]

In [ ]:
c = x
E = (R*T * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))
m = (E.subs(x, cb) - E.subs(x, ca))/(cb-ca)

p1 = symplot(E, (x, 0.7, 0.999), show=False, 
             line_color='black', 
             backend='matplotlib',
             axis_center=[0.69, -98.7],
             xlabel='mole fraction (SiO_2)', ylabel='energy per mole')
for i in range(0, runs):
    A0 = seta0[i]
    A1 = seta1[i]
    Etmp = (R*T * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))
    ptmp = symplot(Etmp, (x, 0.7, 0.999), show=False, line_color='0.8')
    p1.append(ptmp[0])
p1.append(p1[0]) # again, just for overdrawing
# tangent for p1
p2 = symplot(m*(x-ca)+E.subs(x, ca), (x, 0.7, 0.999), show=False, line_color='blue')
p1.append(p2[0])
p1.show()

In [ ]:
R = 0.0083144626181532
B = 12.81
T0 = 923.15
Ts = [T0, T0+150]
A0 = -151.26151
A1 = -85.612615
c = x
E = (R*Ts[0] * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))

p1 = symplot(E, (x, 0.7, 0.999), show=False, 
             line_color='black', 
             backend='matplotlib',
             axis_center=[0.69, -98.5],
             title=f"T={Ts[0]}",
             xlabel='mole fraction (SiO_2)', ylabel='energy per mole')

A0 = 186.0575 - 0.3654*Ts[1]
A1 = 43.7207 - 0.1401*Ts[1]
Etmp = (R*Ts[1] * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))
p2 = symplot(Etmp, (x, 0.7, 0.999), show=False, 
             line_color='0.2',
             axis_center=[0.69, -120.0],
             title=f"T={Ts[1]}",
             xlabel='mole fraction (SiO_2)', ylabel='energy per mole'
            )

plotgrid = sym.plotting.PlotGrid(1, 2, p1, p2, show=False, size=(10, 5))
plotgrid.show()

In [ ]:
R = 0.0083144626181532
B = 12.81
T0 = 923.15
T = T0+150
A0 = -151.26151
A1 = -85.612615
chsimpy.utils.get_miscibility_gap(R=R, T=T, B=B, A0=A0, A1=A1,
                                  xlower=0.85, xupper=0.99, prec=10)

In [ ]:
import glob, os
# solution-17022023-114325-run106.E.csv.bz2
files_E = glob.glob(dirfiles+r'/solution-17022023-114325-run*.E.csv.bz2')

# listE = []

# for filename in files_E:
#     #E = pd.read_csv(f"{dirfiles}/{filename}", sep=',', header=None, compression='bz2').values
#     E = chsimpy.utils.csv_load_matrix(filename)
#     listE.append(E)

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  
# Axes3D import has side effects, it enables using projection='3d' in add_subplot
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = dfraw.A0
y = dfraw.A1
X, Y = np.meshgrid(x, y)
zs = np.array(np.ravel(dfraw.tsep))
Z = zs.reshape(X.shape)

ax.plot_surface(X, Y, Z)